In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split as tts
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

In [2]:
# Get csv
df = pd.read_csv('heart.csv')

df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
# we want to use saga solver in our lin regression model, so normalize
dfNames = list(df.columns.values)
dfTemp = df.values
dfTemp = preprocessing.MinMaxScaler().fit_transform(dfTemp)
df = pd.DataFrame(dfTemp, columns = [dfNames])
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,0.708333,1.0,1.000000,0.481132,0.244292,1.0,0.0,0.603053,0.0,0.370968,0.0,0.0,0.333333,1.0
1,0.166667,1.0,0.666667,0.339623,0.283105,0.0,0.5,0.885496,0.0,0.564516,0.0,0.0,0.666667,1.0
2,0.250000,0.0,0.333333,0.339623,0.178082,0.0,0.0,0.770992,0.0,0.225806,1.0,0.0,0.666667,1.0
3,0.562500,1.0,0.333333,0.245283,0.251142,0.0,0.5,0.816794,0.0,0.129032,1.0,0.0,0.666667,1.0
4,0.583333,0.0,0.000000,0.245283,0.520548,0.0,0.5,0.702290,1.0,0.096774,1.0,0.0,0.666667,1.0


In [4]:
testVariables = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 
                 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
predVariable = ['target']

In [5]:
# # There seems to be a lot of input variables and im scared it might overfit
# # since I'm not sure which variables to eliminate, ill just rfe it.

# rfeModel = LogisticRegression(solver = 'saga', penalty = 'l1')
# rfe = RFE(rfeModel,5)
# rfe = rfe.fit(df[testVariables], np.ravel(df[predVariable]))

In [6]:
# set up train and test sets
x_train, x_test, y_train, y_test = tts(df[testVariables],df[predVariable],train_size=0.95)

In [7]:
# create and run Logistic Regression model
model = LogisticRegression(solver = 'saga', penalty = 'l1')
model.fit(x_train,np.ravel(y_train))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [8]:
# create a prediction of the test cases
y_pred = model.predict(x_test)
y_pred

array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1.])

In [9]:
# dcompare predictions with y_test
model.score(x_test,y_test)

0.9375

In [10]:
# There seems to be a lot of input variables maybe reducing would help
# since i dont know which one works, i will use rfe

rfeModel = LogisticRegression(solver = 'saga', penalty = 'l1')
rfe3 = RFE(rfeModel,3)
rfe4 = RFE(rfeModel,4)
rfe5 = RFE(rfeModel,5)
rfe6 = RFE(rfeModel,6)
rfe7 = RFE(rfeModel,7)
rfe3 = rfe3.fit(x_train, np.ravel(y_train))
rfe4 = rfe4.fit(x_train, np.ravel(y_train))
rfe5 = rfe5.fit(x_train, np.ravel(y_train))
rfe6 = rfe6.fit(x_train, np.ravel(y_train))
rfe7 = rfe7.fit(x_train, np.ravel(y_train))

In [13]:
# seems like 5 or 6 variables is generally a better fit
print('RFE3: {}\nRFE4: {}\nRFE5: {}\nRFE6: {}\nRFE7: {}'
      .format(
       rfe3.score(x_test, y_test),
       rfe4.score(x_test, y_test),
       rfe5.score(x_test, y_test),
       rfe6.score(x_test, y_test),
       rfe7.score(x_test, y_test),))

RFE3: 0.9375
RFE4: 0.8125
RFE5: 1.0
RFE6: 0.9375
RFE7: 1.0
